<img src="https://huggingface.co/allenai/MolmoE-1B-0924/resolve/main/molmo_logo.png" width="500">



Der Code hier ist auf der [Huggingface](https://huggingface.co/allenai/MolmoE-1B-0924) zu finden.




In [1]:
!pip install -q einops torchvision

In [2]:
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig
from PIL import Image
import requests

Zuerst das Model loaden.

Dieser Schritt daurt lange...

In [5]:
# load the processor
processor = AutoProcessor.from_pretrained(
    'allenai/MolmoE-1B-0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

# load the model
model = AutoModelForCausalLM.from_pretrained(
    'allenai/MolmoE-1B-0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPTNeoXTokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.


Als nächster Schritt gibt man Bild und Text dem Model geben.

In diesem Beispiel habe ich dieses Bild verwendet:

<img src="https://iiif.bodleian.ox.ac.uk/iiif/image/790f4240-9750-4818-8a62-4a88e83e7ab2/full/full/0/default.jpg" width="400">



In [6]:
# process the image and text
inputs = processor.process(
    images=[Image.open(requests.get("https://iiif.bodleian.ox.ac.uk/iiif/image/790f4240-9750-4818-8a62-4a88e83e7ab2/full/full/0/default.jpg", stream=True).raw)],
    text="Bitte dieses Bild beschreiben!"
)

# move inputs to the correct device and make a batch of size 1
inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}

# generate output; maximum 200 new tokens; stop generation when <|endoftext|> is generated
output = model.generate_from_batch(
    inputs,
    GenerationConfig(max_new_tokens=200, stop_strings="<|endoftext|>"),
    tokenizer=processor.tokenizer
)

# only get generated tokens; decode them to text
generated_tokens = output[0,inputs['input_ids'].size(1):]
generated_text = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)

# print the generated text
print(generated_text)

 This illustration depicts a medieval scene with two distinct sections. The upper portion shows a royal bedroom where a king and queen are present. The king is wearing a blue robe with a white cape and a crown, while the queen is dressed in a blue robe and a crown as well. They are surrounded by attendants and other people, creating a lively atmosphere.

In the lower section, we see a different scene in what appears to be a castle courtyard. There's a green grassy area with a building in the background. A man is shown opening a door, and inside the building, a unicorn is visible.

The artwork is rich in detail and color, typical of medieval manuscript illustrations. It provides a glimpse into the royal court life and fantastical elements of medieval times, blending reality with mythical creatures.
